In [1]:
from getast import *

In [2]:
asdl_text = open('py3_asdl.simplified.txt').read()
grammar = ASDLGrammar.from_text(asdl_text)
py_code = """pandas.read('file.csv', nrows=100)"""
# get the (domain-specific) python AST of the example Python code snippet
py_ast = ast.parse(py_code)
# convert the python AST into general-purpose ASDL AST used by tranX
asdl_ast = python_ast_to_asdl_ast(py_ast.body[0], grammar)
print('String representation of the ASDL AST: \n%s' % asdl_ast.to_string())
print('Size of the AST: %d' % asdl_ast.size)
# we can also convert the ASDL AST back into Python AST
py_ast_reconstructed = asdl_ast_to_python_ast(asdl_ast, grammar)
src1 = astor.to_source(py_ast).strip()
src2 = astor.to_source(py_ast_reconstructed).strip()
print(asdl_ast.fields)
assert src1 == src2  == "pandas.read('file.csv', nrows=100)"

String representation of the ASDL AST: 
(Expr (expr-value (Call (expr-func (Attribute (expr-value (Name (identifier-id pandas))) (identifier-attr read))) (expr*-args (Str (string-s file.csv))) (keyword*-keywords (keyword (identifier?-arg nrows) (expr-value (Num (object-n 100))))))))
Size of the AST: 12
[Field(expr value)]


In [4]:
tmp_field = asdl_ast.fields[0]

In [10]:
tmp_field.as_value_list

[expr -> Call(expr func, expr* args, keyword* keywords)]

'Expr'

In [21]:
rst_list = []
def to_dict(node,add_list):
    if type(node) == type(" "):
        add_list.append({'name':node,'children':[]})
        return
    this_dict = {'name':node.production.constructor.name,'children':[]}
    add_list.append(this_dict)
    for field in node.fields:
        if field.value is not None:
            for val_node in field.as_value_list:
                if isinstance(field.type, ASDLCompositeType):
                    to_dict(val_node,this_dict['children'])
                else:
                    to_dict(str(val_node),this_dict['children'])    
to_dict(asdl_ast,rst_list)

In [23]:
rst_list[0]

{'name': 'Expr',
 'children': [{'name': 'Call',
   'children': [{'name': 'Attribute',
     'children': [{'name': 'Name',
       'children': [{'name': 'pandas', 'children': []}]},
      {'name': 'read', 'children': []}]},
    {'name': 'Str', 'children': [{'name': 'file.csv', 'children': []}]},
    {'name': 'keyword',
     'children': [{'name': 'nrows', 'children': []},
      {'name': 'Num', 'children': [{'name': '100', 'children': []}]}]}]}]}

In [13]:
x = []
y = {}
x.append(y)
y[0] = 1
print(x[0])

{0: 1}


In [36]:
py_code = '''
for i in range(10):
	print("hello world")
for i in range(10):
	print("hello world")
for i in range(10):
	print("hello world")
a = [100]
print(a)'''

In [37]:
py_ast = ast.parse(py_code)

In [40]:
py_ast.body

In [32]:
asdl_ast = python_ast_to_asdl_ast(py_ast.body[0], grammar)

In [33]:
rst_list = []
def to_dict(node,add_list):
    if type(node) == type(" "):
        add_list.append({'name':node,'children':[]})
        return
    this_dict = {'name':node.production.constructor.name,'children':[]}
    add_list.append(this_dict)
    for field in node.fields:
        if field.value is not None:
            for val_node in field.as_value_list:
                if isinstance(field.type, ASDLCompositeType):
                    to_dict(val_node,this_dict['children'])
                else:
                    to_dict(str(val_node),this_dict['children'])    
to_dict(asdl_ast,rst_list)

In [34]:
rst_list

[{'name': 'Assign',
  'children': [{'name': 'Name', 'children': [{'name': 'a', 'children': []}]},
   {'name': 'List',
    'children': [{'name': 'Num',
      'children': [{'name': '100', 'children': []}]}]}]}]